<a href="https://colab.research.google.com/github/lourencocavalcante/CAP-421-3-Deep-Learning/blob/main/Ativ01_Valdivino_DeepLearning_Louren%C3%A7oCavalcante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[<img src="https://github.com/lourencocavalcante/LogosINPE/blob/main/logoinpe.png?raw=true" width = 500 align="left">](https://www.gov.br/inpe/pt-br)

[<img src="https://github.com/lourencocavalcante/LogosINPE/blob/main/LogoCAP.png?raw=true" width = 300 align="right">](http://www.inpe.br/posgraduacao/cap/)

# **CAP-730 Aprendizado Profundo (*Deep Leraning*)**
## **Detecção de mudanças (change detection - CD) usando rede neural convolucional e a base de dados Modified SYSU-CD**

**Docentes:** *Dr. Valdivino Alexandre de Santiago Junior, Dr. Elcio Hideiti Shiguemori e Dr. Thales Sehn Körti*

**Discente:** *Lourenço José Cavalcante Neto*

---



In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score, f1_score
import torch.nn.functional as F

In [ ]:
!pip install torch torchvision matplotlib > /dev/null 2>&1

!pip install efficientnet_pytorch segmentation-models-pytorch > /dev/null 2>&1

!pip install -q kaggle > /dev/null 2>&1


In [ ]:
%%bash

# Define o caminho do arquivo para download
output_dir="/kaggle/working/modified-sysu-cd.zip"

# Verifica se o arquivo já foi baixado
if [ -e "$output_dir" ]; then
    echo "O arquivo modified-sysu-cd.zip já está presente."
else
    # Realiza o download caso o arquivo não exista
    kaggle datasets download valdivinosantiago/modified-sysu-cd
fi


O arquivo modified-sysu-cd.zip já está presente.


In [ ]:
import os
import zipfile

# Define os caminhos do arquivo compactado e do destino da extração
zip_path = "/kaggle/working/modified-sysu-cd.zip"
extract_to = "/kaggle/working/sample_data"

# Verifica se o diretório de destino já foi criado
if not os.path.exists(extract_to):
    # Extrai o conteúdo do arquivo zip, se ainda não extraído
    with zipfile.ZipFile(zip_path, 'r') as archive:
        archive.extractall(extract_to)
else:
    print(f"O diretório {extract_to} já está disponível. Nenhuma ação necessária.")


O diretório /kaggle/working/sample_data já está disponível. Nenhuma ação necessária.


In [ ]:
#!/bin/bash

# Baixa o dataset especificado usando o comando Kaggle
!kaggle datasets download -d valdivinosantiago/modified-sysu-cd


Dataset URL: https://www.kaggle.com/datasets/valdivinosantiago/modified-sysu-cd
License(s): other
modified-sysu-cd.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from torchvision import transforms

class CustomSYSUCDDataset(Dataset):

    def __init__(self, base_dir, image_transform=None, mask_transform=None):
        self.base_dir = base_dir
        self.image_transform = image_transform
        self.mask_transform = mask_transform

        # Listar arquivos válidos nos diretórios
        self.img_files = sorted([file for file in os.listdir(os.path.join(base_dir, 'time1'))
                                 if os.path.isfile(os.path.join(base_dir, 'time1', file))])
        self.mask_files = sorted([file for file in os.listdir(os.path.join(base_dir, 'label'))
                                  if os.path.isfile(os.path.join(base_dir, 'label', file))])

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, index):
        # Caminhos completos para as imagens de T1, T2 e máscaras
        img_t1_path = os.path.join(self.base_dir, 'time1', self.img_files[index])
        img_t2_path = os.path.join(self.base_dir, 'time2', self.img_files[index])
        mask_path = os.path.join(self.base_dir, 'label', self.mask_files[index])

        # Carregar as imagens
        img_t1 = Image.open(img_t1_path).convert('RGB')
        img_t2 = Image.open(img_t2_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')

        # Aplicar as transformações, se definidas
        if self.image_transform:
            img_t1 = self.image_transform(img_t1)
            img_t2 = self.image_transform(img_t2)
        if self.mask_transform:
            mask = self.mask_transform(mask)

        return img_t1, img_t2, mask


# Definir transformações
img_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

mask_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Definir caminhos dos dados
train_dir = '/kaggle/working/sample_data/trainmod'
val_dir = '/kaggle/working/sample_data/valmod'
test_dir = '/kaggle/working/sample_data/testmod'

# Inicializar datasets
train_data = CustomSYSUCDDataset(train_dir, image_transform=img_transforms, mask_transform=mask_transforms)
val_data = CustomSYSUCDDataset(val_dir, image_transform=img_transforms, mask_transform=mask_transforms)
test_data = CustomSYSUCDDataset(test_dir, image_transform=img_transforms, mask_transform=mask_transforms)

batch_size = 6

# Criar DataLoaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)


In [ ]:
def display_batch_shapes(loader):
    for batch_idx, (image_t1, image_t2, label) in enumerate(loader):
        print(f'Lote {batch_idx + 1}:')
        print(f'Forma de image_t1: {image_t1.shape}')
        print(f'Forma de image_t2: {image_t2.shape}')
        print(f'Forma de label: {label.shape}')
        print('---')

        if batch_idx == 1:
            break


print("Formas de saída do conjunto de treinamento:")
display_batch_shapes(train_loader)

print("Formas de saída do conjunto de validação:")
display_batch_shapes(val_loader)

print("Formas de saída do conjunto de teste:")
display_batch_shapes(test_loader)


Formas de saída do conjunto de treinamento:
Lote 1:
Forma de image_t1: torch.Size([6, 3, 256, 256])
Forma de image_t2: torch.Size([6, 3, 256, 256])
Forma de label: torch.Size([6, 1, 256, 256])
---
Lote 2:
Forma de image_t1: torch.Size([6, 3, 256, 256])
Forma de image_t2: torch.Size([6, 3, 256, 256])
Forma de label: torch.Size([6, 1, 256, 256])
---
Formas de saída do conjunto de validação:
Lote 1:
Forma de image_t1: torch.Size([6, 3, 256, 256])
Forma de image_t2: torch.Size([6, 3, 256, 256])
Forma de label: torch.Size([6, 1, 256, 256])
---
Lote 2:
Forma de image_t1: torch.Size([6, 3, 256, 256])
Forma de image_t2: torch.Size([6, 3, 256, 256])
Forma de label: torch.Size([6, 1, 256, 256])
---
Formas de saída do conjunto de teste:
Lote 1:
Forma de image_t1: torch.Size([6, 3, 256, 256])
Forma de image_t2: torch.Size([6, 3, 256, 256])
Forma de label: torch.Size([6, 1, 256, 256])
---
Lote 2:
Forma de image_t1: torch.Size([6, 3, 256, 256])
Forma de image_t2: torch.Size([6, 3, 256, 256])
Forma d

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm
from torchvision import models

# U-Net com encoder ResNet34 pré-treinado
class ResNet34UNet(nn.Module):
    def __init__(self, in_channels=6, out_classes=1):
        super(ResNet34UNet, self).__init__()

        # ResNet34 pré-treinada como encoder
        backbone = models.resnet34(pretrained=True)
        backbone.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Reutilização do encoder da ResNet34, removendo camadas finais
        self.encoder = nn.Sequential(*list(backbone.children())[:-2])

        # Camadas de upsampling e decodificação
        self.upconv1 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.relu1 = nn.ReLU(inplace=True)

        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.relu2 = nn.ReLU(inplace=True)

        # Camada final para ajuste de saída
        self.upsample = nn.Upsample(size=(256, 256), mode='bilinear', align_corners=True)
        self.final_layer = nn.Conv2d(128, out_classes, kernel_size=1)

    def forward(self, input_data):
        encoded = self.encoder(input_data)
        decoded = self.relu1(self.upconv1(encoded))
        decoded = self.relu2(self.upconv2(decoded))
        upsampled = self.upsample(decoded)
        output = self.final_layer(upsampled)

        return output


# Função para calcular a precisão binária
def binary_accuracy(predictions, labels):
    predictions = torch.sigmoid(predictions)  # Convertendo logits para probabilidade
    predictions = (predictions > 0.5).float()  # Binarizando usando threshold de 0.5
    matches = (predictions == labels).float().sum()
    accuracy = matches / labels.numel()  # Total de elementos
    return accuracy.item()


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Definir a classe do Dataset
class ModifiedSYSUCDDataset(Dataset):
    def __init__(self, root_dir, transform=None, label_transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.label_transform = label_transform

        self.image_pairs = sorted([f for f in os.listdir(os.path.join(root_dir, 'time1')) if os.path.isfile(os.path.join(root_dir, 'time1', f))])
        self.masks = sorted([f for f in os.listdir(os.path.join(root_dir, 'label')) if os.path.isfile(os.path.join(root_dir, 'label', f))])

    def __len__(self):
        return len(self.image_pairs)

    def __getitem__(self, idx):
        t1_path = os.path.join(self.root_dir, 'time1', self.image_pairs[idx])
        t2_path = os.path.join(self.root_dir, 'time2', self.image_pairs[idx])
        mask_path = os.path.join(self.root_dir, 'label', self.masks[idx])

        t1_image = Image.open(t1_path).convert('RGB')
        t2_image = Image.open(t2_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')

        if self.transform:
            t1_image = self.transform(t1_image)
            t2_image = self.transform(t2_image)
        if self.label_transform:
            mask = self.label_transform(mask)

        return t1_image, t2_image, mask

# Definir transformações para as imagens e máscaras
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

label_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Caminho do dataset
root_dir_train = '/kaggle/working/sample_data/trainmod'
root_dir_val = '/kaggle/working/sample_data/valmod'

# Criar datasets para treino e validação
train_dataset = ModifiedSYSUCDDataset(root_dir_train, transform=transform, label_transform=label_transform)
valid_dataset = ModifiedSYSUCDDataset(root_dir_val, transform=transform, label_transform=label_transform)

# Definir DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=6, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=6, shuffle=False)

# Definir o modelo U-Net com ResNet34 como encoder
class ResNet34UNet(nn.Module):
    def __init__(self, in_channels=6, out_classes=1):
        super(ResNet34UNet, self).__init__()

        # Carregar a ResNet34 pré-treinada
        resnet34 = models.resnet34(pretrained=True)
        resnet34.conv1 = nn.Conv2d(in_channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

        # Usar o resto da ResNet34 (encoder)
        self.encoder = nn.Sequential(*list(resnet34.children())[:-2])  # Excluindo as camadas finais da ResNet

        # Camadas de decodificação
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2, padding=0),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2, padding=0),
            nn.ReLU(inplace=True),
        )

        # Upsampling final para garantir que a saída tenha o tamanho exato (256, 256)
        self.final_upsample = nn.Upsample(size=(256, 256), mode='bilinear', align_corners=True)
        self.final_conv = nn.Conv2d(128, out_classes, kernel_size=1)

    def forward(self, x):
        x = self.encoder(x)
        # Passar pelo decoder
        x = self.decoder(x)
        x = self.final_upsample(x)
        x = self.final_conv(x)

        return x

# Função para calcular a acurácia binária
def binary_accuracy(preds, targets):
    preds = torch.sigmoid(preds)  # Aplicar sigmoid para obter probabilidade
    preds = (preds > 0.5).float()  # Binarizar com threshold 0.5
    correct = (preds == targets).float().sum()
    accuracy = correct / (targets.numel())  # Total de elementos
    return accuracy.item()

# Função de treinamento e validação
def train_and_validate(model, train_loader, valid_loader, num_epochs=25, model_save_path='/kaggle/working/best_model.pth'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Definindo a função de perda e o otimizador
    loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([2.0]).to(device))
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    highest_valid_accuracy = 0.0  # Melhor acurácia de validação registrada

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        total_train_accuracy = 0

        # Laço de treinamento
        for img_t1, img_t2, target_mask in tqdm(train_loader):
            # Concatenar as imagens t1 e t2 ao longo do canal
            input_images = torch.cat((img_t1, img_t2), dim=1).to(device)
            target_mask = target_mask.to(device)

            optimizer.zero_grad()
            predictions = model(input_images)
            loss = loss_fn(predictions, target_mask)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()
            total_train_accuracy += binary_accuracy(predictions, target_mask)

        # Calcular métricas de treinamento
        avg_train_loss = total_train_loss / len(train_loader)
        avg_train_accuracy = total_train_accuracy / len(train_loader)

        # Fase de validação
        model.eval()
        total_valid_loss = 0
        total_valid_accuracy = 0

        with torch.no_grad():
            for img_t1, img_t2, target_mask in valid_loader:
                input_images = torch.cat((img_t1, img_t2), dim=1).to(device)
                target_mask = target_mask.to(device)

                predictions = model(input_images)
                loss = loss_fn(predictions, target_mask)

                total_valid_loss += loss.item()
                total_valid_accuracy += binary_accuracy(predictions, target_mask)

        # Calcular métricas de validação
        avg_valid_loss = total_valid_loss / len(valid_loader)
        avg_valid_accuracy = total_valid_accuracy / len(valid_loader)

        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {avg_train_loss:.4f}, Train Accuracy: {avg_train_accuracy:.4f}, "
              f"Valid Loss: {avg_valid_loss:.4f}, Valid Accuracy: {avg_valid_accuracy:.4f}")

        # Salvar o modelo com a melhor acurácia de validação
        if avg_valid_accuracy > highest_valid_accuracy:
            highest_valid_accuracy = avg_valid_accuracy
            torch.save(model.state_dict(), model_save_path)
            print(f"Novo modelo salvo com acurácia de validação: {highest_valid_accuracy:.4f}")

    print(f"Treinamento concluído. Melhor acurácia de validação: {highest_valid_accuracy:.4f}")

# Instanciar o modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet34UNet(in_channels=6, out_classes=1).to(device)

# Treinamento do modelo
train_and_validate(model, train_loader, valid_loader, num_epochs=50)


100%|██████████| 1000/1000 [01:41<00:00,  9.89it/s]


Epoch [1/50], Train Loss: 0.4808, Train Accuracy: 0.8450, Valid Loss: 0.4317, Valid Accuracy: 0.8443
Novo modelo salvo com acurácia de validação: 0.8443


100%|██████████| 1000/1000 [01:41<00:00,  9.84it/s]


Epoch [2/50], Train Loss: 0.4111, Train Accuracy: 0.8689, Valid Loss: 0.6336, Valid Accuracy: 0.8633
Novo modelo salvo com acurácia de validação: 0.8633


100%|██████████| 1000/1000 [01:43<00:00,  9.68it/s]


...

Epoch [48/50], Train Loss: 0.0371, Train Accuracy: 0.9885, Valid Loss: 0.7835, Valid Accuracy: 0.8979


100%|██████████| 1000/1000 [01:37<00:00, 10.21it/s]


Epoch [49/50], Train Loss: 0.0289, Train Accuracy: 0.9908, Valid Loss: 0.8693, Valid Accuracy: 0.9030


100%|██████████| 1000/1000 [01:38<00:00, 10.20it/s]


Epoch [50/50], Train Loss: 0.0311, Train Accuracy: 0.9904, Valid Loss: 0.9665, Valid Accuracy: 0.8967
Treinamento concluído. Melhor acurácia de validação: 0.9065


In [ ]:
import os
os.chdir('/kaggle/working')
!mkdir 'ChangeDetection_Valdivino'

mkdir: cannot create directory 'ChangeDetection_Valdivino': File exists


In [ ]:
import time
import torch
import torch.nn as nn
from sklearn.metrics import f1_score
import numpy as np
import torch.optim as optim
from tqdm import tqdm
import torchvision.models as models

# Definir uma versão modificada do modelo U-Net com ResNet34 como backbone
class ModifiedUNet(nn.Module):
    def __init__(self, input_channels=6, output_classes=1):
        super(ModifiedUNet, self).__init__()

        # Carregar ResNet34 com pesos pré-treinados
        base_model = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)
        base_model.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Usar a parte inicial da ResNet (encoder)
        self.encoder = nn.Sequential(*list(base_model.children())[:-2])

        # Camadas do decoder (para reconstruir as dimensões da imagem)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2, padding=0),  # Expande para (128, 128)
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2, padding=0),  # Expande para (256, 256)
            nn.ReLU(inplace=True),
        )

        # Ajuste final de upsampling
        self.upsample = nn.Upsample(size=(256, 256), mode='bilinear', align_corners=True)
        self.final_conv = nn.Conv2d(128, output_classes, kernel_size=1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        x = self.upsample(x)
        x = self.final_conv(x)
        return x

# Função para calcular o IoU (Intersection over Union)
def compute_iou(predictions, targets, threshold=0.5):
    # Binarizar as predições e os alvos
    predictions = predictions > threshold
    targets = targets > 0.5

    # Flatten para comparação
    predictions = predictions.view(predictions.size(0), -1)
    targets = targets.view(targets.size(0), -1)

    intersection = (predictions * targets).sum(dim=1)
    union = (predictions + targets).sum(dim=1)

    # Calcular a média do IoU para o batch
    iou = (intersection + 1e-6) / (union + 1e-6)
    return iou.mean().item()

# Função para avaliar o modelo no conjunto de teste
def model_evaluation(model, dataloader, device='cuda'):
    model.eval()

    all_preds = []
    all_true_masks = []

    total_time = 0
    batch_count = 0

    with torch.no_grad():
        for img_t1, img_t2, mask in dataloader:
            # Concatenar as imagens de diferentes tempos
            input_images = torch.cat((img_t1, img_t2), dim=1).to(device)
            true_mask = mask.to(device)

            start = time.time()

            # Obter a predição do modelo
            outputs = model(input_images)

            end = time.time()
            total_time += (end - start) * 1000
            batch_count += 1

            # Armazenar predições e máscaras verdadeiras para métricas
            all_preds.append(torch.sigmoid(outputs).cpu().numpy())
            all_true_masks.append(true_mask.cpu().numpy())

    # Calcular a latência média
    avg_latency = total_time / batch_count

    # Flatten as predições e as máscaras
    all_preds = np.concatenate(all_preds, axis=0).flatten()
    all_true_masks = np.concatenate(all_true_masks, axis=0).flatten()

    # Calcular o IoU
    iou = compute_iou(torch.tensor(all_preds > 0.5), torch.tensor(all_true_masks))
    print(f"IoU: {iou:.4f}")

    # Calcular o F1-score
    f1 = f1_score(all_true_masks, all_preds > 0.5)

    # Exibir métricas
    print(f"F1-score: {f1:.4f}")
    print(f"Latência: {total_time:.4f} ms")
    print(f"Latência média: {avg_latency:.4f} ms por batch") total_time

    return f1, iou, avg_latency

# Carregar o modelo treinado
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = ModifiedUNet(input_channels=6, output_classes=1).to(device)
model.load_state_dict(torch.load('/kaggle/working/best_model.pth'))

# Avaliar o modelo no conjunto de teste
f1, iou, avg_latency = model_evaluation(model, test_loader, device=device)


/tmp/ipykernel_30/3763316527.py:110: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/working/best_model.pth'))


IoU: 0.9076
F1-score: 0.7567
Latência média: 5.3940 ms por batch
